# Text generation using tensor2tensor on Cloud AI Platform

This notebook illustrates the use of [tensor2tensor](https://github.com/tensorflow/tensor2tensor) library to do from-scratch, distributed training of a training model. Then, the trained model is used to complete new poems.

## Install tensor2tensor, and specify Google Cloud Platform project and bucket

Install the necessary packages. tensor2tensor will give us the Transformer model. Project Gutenberg gives us access to historical poems.

**p.s.** Note that this notebook uses Python 2 because Project Gutenberg relies on BSD-DB which was deprecated in Python 3 and removed from the standard library. tensor2tensor itself can be used on Python 3.

In [ ]:
%%bash
pip freeze | grep tensor

In [ ]:
%%bash
pip install tensor2tensor=1.13.1 tensorflow=1.13.1 tensorflow-serving-api=1.13 gutenberg
pip install tensorflow_hub

# install from sou
#git clone https://github.com/tensorflow/tensor2tensor.git
#cd tensor2tensor
#yes | pip install --user -e .

In [ ]:
%%bash
pip freeze | grep tensor

In [ ]:
import os
PROJECT = "cloud-training-demos" # Replace with your Project ID
BUCKET = "cloud-training-demos-ml" # Replace with your bucket name
REGION = "us-central1" # Replace with your bucket region

# this is what this notebook is demonstrating
PROBLEM = "poetry_line_problem"

# for bash
os.environ["PROJECT"] = PROJECT
os.environ["BUCKET"] = BUCKET
os.environ["REGION"] = REGION
os.environ["PROBLEM"] = PROBLEM

# os.environ["PATH"] = os.environ["PATH"] + ":" + os.getcwd() + "/tensor2tensor/tensor2tensor/bin"

In [ ]:
%bash
gcloud config set project $PROJECT
gcloud config set compute/region $REGION

## Download data

We will get some [poetry anthologies](https://www.gutenberg.org/wiki/Poetry_(Bookshelf)) from Project Gutenberg.

In [ ]:
%%bash
rm -rf data/poetry
mkdir -p data/poetry

In [ ]:
from gutenberg.acquire import load_etext
from gutenberg.cleanup import strip_headers
import re

books = [
    # bookid, skip N lines, title
    (26715, 1000, "Victorian songs"),
    (30235, 580, "Baldwin collection"),
    (35402, 710, "Swinburne collection"),
    (574, 15, "Blake"),
    (1304, 172, "Bulchevys collection"),
    (19221, 223, "Palgrave-Pearse collection"),
    (15553, 522, "Knowles collection")
]

with open("data/poetry/raw.txt", "w") as ofp:
    lineno = 0
    for (id_nr, toskip, title) in books:
        startline = lineno
        text = strip_headers(load_etext(id_nr)).strip()
        lines = text.split("\n")[toskip:]
        # any line that is all upper case is a title or author name
        # also don't want any lines with years (numbers)
        for line in lines:
            if (len(line) > 0
                and line.upper() != line
                and not re.match(".*[0-9]+.*", line)
                and len(line) < 50
                ):
                cleaned = re.sub("[^a-z\"\-]+", " ", line.strip().lower())
                ofp.write(cleaned)
                ofp.write("\n")
                lineno += 1
            else:
                ofp.write("\n")
        print("Wrote lines {} to {} from {}".format(startline, lineno, title))

In [ ]:
!wc -l data/poetry/*.txt